In [1]:
from __future__ import print_function
import os
data_path = ['data']

import pandas as pd
import numpy as np

filepath = os.sep.join(data_path + ['Orange_Telecom_Churn_Data.csv'])
data = pd.read_csv(filepath)

#상관없어보이는 특징 제거
data.drop(['account_length','state', 'area_code', 'phone_number', 'total_day_calls'], axis=1, inplace=True)

from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()

for col in ['intl_plan', 'voice_mail_plan', 'churned']:
    data[col] = lb.fit_transform(data[col])
    
import warnings
warnings.filterwarnings('ignore', module='sklearn')

from sklearn.preprocessing import MinMaxScaler

msc = MinMaxScaler()

data = pd.DataFrame(msc.fit_transform(data),  # this is an np.array, not a dataframe.
                    columns=data.columns)
data.churned.value_counts()

#data.head(10).T

0.0    4293
1.0     707
Name: churned, dtype: int64

In [2]:
#total_eve_charge,, total_intl_minutes,total_night_minutes, number_vmail_messages, total_day_minutes, total_day_charge, total_eve_minutes, total_eve_calls, total_night_calls, total_night_charge, total_intl_charge, 
#total_day_calls,
#total_intl_calls, number_customer_service_calls 매우 중요한 요소

In [3]:
churned_indices = data[data.churned == 0.0].index
non_churned_indices = data[data.churned == 1.0].index

churned_sample = data.loc[churned_indices]
shuffled_sample = churned_sample.reindex(np.random.permutation(churned_sample.index))
non_churned_sample = data.loc[non_churned_indices]

####
shuffled_sample2 = non_churned_sample.reindex(np.random.permutation(non_churned_sample.index))
#s7 = shuffled_sample2[607:707]
#s6 = shuffled_sample[3536:4293]
#s7 = pd.concat([s7, s6])
####

f1 = shuffled_sample2[:606]

s1 = shuffled_sample[:707]
s1 = pd.concat([s1, f1])
s2 = shuffled_sample[708:1414]
s2 = pd.concat([s2, f1])
s3 = shuffled_sample[1415:2121]
s3 = pd.concat([s3, f1])
s4 = shuffled_sample[2122:2828]
s4 = pd.concat([s4, f1])
s5 = shuffled_sample[2829:3535]
s5 = pd.concat([s5, f1])

f2 = shuffled_sample2[607:707]
s6 = shuffled_sample[3536:4293]
s6 = pd.concat([s6, f2])

In [4]:
#data.head(10).T

In [5]:
x_cols = [x for x in data.columns if x != 'churned']

X_data = data[x_cols]
y_data = data['churned']

In [6]:
x_cols = [x for x in s1.columns if x != 'churned']
X_data_1 = s1[x_cols]
y_data_1 = s1['churned']

x_cols = [x for x in s2.columns if x != 'churned']
X_data_2 = s2[x_cols]
y_data_2 = s2['churned']

x_cols = [x for x in s3.columns if x != 'churned']
X_data_3 = s3[x_cols]
y_data_3 = s3['churned']

x_cols = [x for x in s4.columns if x != 'churned']
X_data_4 = s4[x_cols]
y_data_4 = s4['churned']

x_cols = [x for x in s5.columns if x != 'churned']
X_data_5 = s5[x_cols]
y_data_5 = s5['churned']

x_cols = [x for x in s6.columns if x != 'churned']
X_data_6 = s6[x_cols]
y_data_6 = s6['churned']

x_cols = [x for x in data.columns if x != 'churned']
X_data = data[x_cols]
y_data = data['churned']

In [7]:
from sklearn.neighbors import KNeighborsClassifier

knn1 = KNeighborsClassifier(n_neighbors=3, p=2)
knn2 = KNeighborsClassifier(n_neighbors=3, p=2)
knn3 = KNeighborsClassifier(n_neighbors=3, p=2)
knn4 = KNeighborsClassifier(n_neighbors=3, p=2)
knn5 = KNeighborsClassifier(n_neighbors=3, p=2)
#knn6 = KNeighborsClassifier(n_neighbors=3, p=2)
knn1 = knn1.fit(X_data_1, y_data_1)
knn2 = knn2.fit(X_data_2, y_data_2)
knn3 = knn3.fit(X_data_3, y_data_3)
knn4 = knn4.fit(X_data_4, y_data_4)
knn5 = knn5.fit(X_data_5, y_data_5)
#knn6 = knn6.fit(X_data_6, y_data_6)

y_pred1 = knn1.predict(X_data_6)
y_pred2 = knn2.predict(X_data_6)
y_pred3 = knn3.predict(X_data_6)
y_pred4 = knn4.predict(X_data_6)
y_pred5 = knn5.predict(X_data_6)
#y_pred6 = knn6.predict(X_data)

y_pred = y_pred1 + y_pred2 + y_pred3 + y_pred4 + y_pred5

#for i in y_pred:
#    print(i)


In [8]:
type(y_pred)

numpy.ndarray

In [9]:
with np.nditer(y_pred, op_flags=['readwrite']) as it:
    for i in it:
        if i>=5:
            i[...]=1
        else:
            i[...]=0.0
        
#for i in y_pred:
#    print(i)
        

In [10]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_data_6, y_pred)

array([[714,  43],
       [ 45,  55]])

In [11]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
metrics = list()

precision, recall, fscore, _ = score(y_data_6, y_pred, average='weighted')
accuracy = accuracy_score(y_data_6, y_pred)

metrics.append(pd.Series({'precision':precision, 'recall':recall, 
                        'fscore':fscore, 'accuracy':accuracy}, 
                        name='scores'))

metrics = pd.concat(metrics, axis=1)

metrics

,scores
precision,0.896431
recall,0.897316
fscore,0.896865
accuracy,0.897316
